In [16]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import pandas as pd
import plotly.express as px
from dash import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import sys
import numpy as np
from pymongo import MongoClient
from bson.json_util import dumps
import base64

#### DONE #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelterCrud import AnimalShelter

In [17]:
###########################
# Data Manipulation / Model
###########################
# DONE change for your username and password and CRUD Python module name
username = "aacuser"
password = "French"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))


In [18]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Kaitlyn French Project Two')

#DONE Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

In [19]:
#DONE Place the HTML image tag in the line below into the app.layout code according to your design
#DONE Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Kaitlyn French CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#DONE Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': html.Div(['Water Rescue'], style={'color': 'Blue', 'font-size':20}),
             'value': 'WR'},
            {'label': html.Div(['Mountain or Wilderness Rescue'], style={'color': 'Brown', 'font-size':20}),
             'value': 'MWR'},
            {'label': html.Div(['Disaster Rescue or Individual Tracking'], style={'color': 'DarkOrange', 'font-size':20}),
             'value': 'DRIT'},
            {'label': html.Div(['Reset'], style={'color':'Red', 'font-size':20}),
             'value':'RESET'}
        ],
        value='RESET',
        labelStyle={'display':'inline-block', 'margin-left':'20px'}
        )
    ),
   
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#DONE: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

In [20]:
#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(selected_filter):
### DONE Add code to filter interactive data table with MongoDB queries

        if (selected_filter == 'WR'):
            df = pd.DataFrame(list(shelter.read(
                {
                    "animal_type":"Dog",
                    "breed":{"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
                    "sex_upon_outcome":"Intact Female",
                    "age_upon_outcome_in_weeks":{"$gte":26},
                    "age_upon_outcome_in_weeks":{"$lte":156}
                })
        ))
        elif (selected_filter == 'MWR'):
            #Grazioso breeds and ages
            df = pd.DataFrame(list(shelter.read(
            {
                    "animal_type":"Dog",
                    "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
                    "sex_upon_outcome":"Intact Male",
                    "age_upon_outcome_in_weeks":{"$gte":26},
                    "age_upon_outcome_in_weeks":{"$lte":156}
            })
        ))
        #adjusts the read request for the desired dog type and status
        elif (selected_filter == 'DRIT'):
            #breeds and ages
            df = pd.DataFrame(list(shelter.read(
            {
                    "animal_type":"Dog",
                    "breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
                    "age_upon_outcome_in_weeks":{"$gte":20},
                    "age_upon_outcome_in_weeks":{"$lte":300}
            })
        ))
        #resets the search no filter
        elif (selected_filter == 'RESET'):
            df = pd.DataFrame.from_records(shelter.readAll({}))

        
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        return (data, columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'),
    Input('datatable-id', "derived_viewport_selected_rows"),
    Input('datatable-id', 'active_cell')]
)
def update_styles(selected_columns, selected_rows, active_cell):
    if active_cell is not None:
        style = [{
                    'if': {'row_index': active_cell['row'] },
                    'background_color': '#90EE90'
                }]
                                   
    else:
        style = [{
                    'if': {'row_index': i},
                     'background_color': '#90EE90'
                } for i in selected_rows]
                                   
    return (style +
                [{
                     'if': { 'column_id': i},
                    'background_color': '#90EE90'
                } for i in selected_columns]
           )

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    ### DONE ####
    dff = pd.DataFrame.from_dict(viewData)
                                   
    # add code for chart of your choice (e.g. pie chart) #
    fig = px.pie(
        dff,
        names = 'breed',
        title = 'Grazioso Salvare Animal Breeds Pie Chart'
    )
    return [dcc.Graph(figure = fig)]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_viewport_selected_rows"),
     Input('datatable-id', "active_cell")])
                                   
def update_map(viewData, selected_rows, active_cell):
#DONE: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
    dff = pd.DataFrame.from_dict(viewData)
    
    if active_cell is not None:
        row = active_cell['row']
    else:
        row = selected_rows[0]

    lat = dff.loc[row, 'location_lat']
    long = dff.loc[row,'location_long']
    name = dff.loc[row,'name']
    breed = dff.loc[row,'breed']
    animal = dff.loc[row, 'animal_type']
    age = dff.loc[row, 'age_upon_outcome']
    
    if name =="":
        name = "No Name"
    # Austin TX is at [30.75, -97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ]) 
        ])
    ]

app